In [1]:
import pandas as pd
import matplotlib as mp
import numpy as np
from datetime import timedelta
import datetime as dt

%matplotlib inline

#Root de archivos de datos
root_dir = '../data/'

In [2]:
#Optimizado para menos memoria
auction_dtypes = {
    'ref_type_id': np.int64,
    'device_id': np.int64,
}

auctions = pd.read_csv(root_dir + 'auctions.csv.gzip',
                       compression = 'gzip',
                       dtype = auction_dtypes,
                       usecols = list(auction_dtypes.keys()))
auctions.rename({'device_id':'ref_hash',
                 'ref_type_id':'ref_type'}, inplace=True, axis='columns')

In [3]:
auctions.ref_type.value_counts()

1    40330743
7     7078785
Name: ref_type, dtype: int64

In [4]:
devices = auctions[['ref_type','ref_hash']]

In [5]:
installs_dtypes = {
    'ref_type': np.int64,
    'ref_hash': np.int64, 
}
install_cols = list(installs_dtypes.keys())
installs = pd.read_csv(root_dir + 'installs.csv.gzip', 
                       compression='gzip', 
                       usecols=install_cols,
                       dtype= installs_dtypes)

In [6]:
installs.ref_type.value_counts()

1891515180541284343    398906
1494519392962156891     82605
Name: ref_type, dtype: int64

In [9]:
clicks_dtypes = {
    'ref_type':np.int64, 
    'ref_hash':np.int64
}
clicks = pd.read_csv(root_dir + 'clicks.csv.gzip', 
                     compression='gzip', 
                       usecols=list(clicks_dtypes.keys()),
                       dtype= clicks_dtypes)

In [10]:
events_dtypes = {
    'ref_type': np.int64,
    'ref_hash': np.int64
}
events_cols = list(events_dtypes.keys())
events = pd.read_csv(root_dir + 'events.csv.gzip', 
                     compression='gzip',
                     dtype=events_dtypes,
                     usecols=events_cols)

In [11]:
for x in [installs,clicks,events]:
    x.loc[x['ref_type']== 1891515180541284343,'ref_type'] = 1
    x.loc[x['ref_type'] == 1494519392962156891,'ref_type'] = 7
    print(x['ref_type'].value_counts())
    devices = devices.append(x[['ref_type','ref_hash']])
devices = devices.drop_duplicates()

1    398906
7     82605
Name: ref_type, dtype: int64
1    60492
7     3804
Name: ref_type, dtype: int64
1    6421584
7    1322997
Name: ref_type, dtype: int64


In [16]:
dev_7 = devices.set_index('ref_type').loc[7,'ref_hash']
hash_to_drop = dev_7.loc[dev_7.isin(devices.set_index('ref_type').loc[1,'ref_hash'].unique())].unique()

In [25]:
uniques = devices.loc[~devices['ref_hash'].isin(hash_to_drop),'ref_hash']

In [26]:
uniques.to_csv(root_dir + 'unique_hashes.csv', header=True,index=False)